# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [15]:
! git clone https://github.com/zakariaabou/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 21), reused 20 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.41 MiB | 16.37 MiB/s, done.
Resolving deltas: 100% (21/21), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [16]:
mySeed = 333

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [18]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

In [19]:
# Assuming train_set and test_set are defined as in the provided code.

# Extract data from the training set
X_train = torch.tensor(train_set['inputs'], dtype=torch.float32)
y_train = torch.tensor(train_set['targets'], dtype=torch.float32).reshape(-1, 1)  # Reshape to column vector

# Add a column of ones for the intercept term (theta_0)
X_train = torch.cat((torch.ones(X_train.shape[0], 1), X_train), dim=1)


# Calculate the coefficients using the normal equation (least squares)
theta_hat = torch.linalg.lstsq(X_train, y_train).solution

# Print the estimated coefficients
print("Estimated coefficients (theta_hat):\n", theta_hat)

Estimated coefficients (theta_hat):
 tensor([[16.5580],
        [ 3.2730],
        [ 3.2948],
        [ 6.6870]])


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

In [23]:
import torch.nn as nn
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(3, 1)

    def forward(self, x):
        return self.fc(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [25]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        optimizer.zero_grad()  # Réinitialisation des gradients
        outputs = mySimpleNet(batch_inputs)  # Prédictions du modèle
        loss = criterion(outputs, batch_targets)  # Calcul de la perte
        loss.backward()  # Calcul des gradients
        optimizer.step()  # Mise à jour des paramètres du modèle
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


Epoch 0, Loss: 300.1123962402344
Epoch 10, Loss: 58.33413314819336
Epoch 20, Loss: 36.03379821777344
Epoch 30, Loss: 46.22721481323242
Epoch 40, Loss: 29.654056549072266
Epoch 50, Loss: 33.96439743041992
Epoch 60, Loss: 37.58482360839844
Epoch 70, Loss: 24.910655975341797
Epoch 80, Loss: 28.84027099609375
Epoch 90, Loss: 28.49748420715332
Epoch 100, Loss: 41.96553421020508
Epoch 110, Loss: 31.527070999145508
Epoch 120, Loss: 40.82662582397461
Epoch 130, Loss: 45.983551025390625
Epoch 140, Loss: 27.101842880249023
Epoch 150, Loss: 34.97255325317383
Epoch 160, Loss: 28.91143035888672
Epoch 170, Loss: 33.78229522705078
Epoch 180, Loss: 37.98525619506836
Epoch 190, Loss: 37.588417053222656
Epoch 200, Loss: 33.54921340942383
Epoch 210, Loss: 27.253435134887695
Epoch 220, Loss: 34.91481018066406
Epoch 230, Loss: 43.703548431396484
Epoch 240, Loss: 41.667137145996094
Epoch 250, Loss: 36.991493225097656
Epoch 260, Loss: 34.33574676513672
Epoch 270, Loss: 40.04707336425781
Epoch 280, Loss: 38.6

**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [31]:
# Extraction des estimations des θk
theta_estimated_bias = mySimpleNet.fc.bias.data
theta_estimated_weights = mySimpleNet.fc.weight.data.flatten()

# Combinaison des biais et des poids
theta_estimated = np.concatenate((theta_estimated_bias, theta_estimated_weights))
print("Estimations des θk:", theta_estimated)

Estimations des θk: [19.918228    0.02555635  0.02251558  0.06640711]


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [36]:
# Prepare test data
X_test = torch.tensor(test_set['inputs'], dtype=torch.float32)
y_test = torch.tensor(test_set['targets'], dtype=torch.float32).reshape(-1, 1)
X_test = torch.cat((torch.ones(X_test.shape[0], 1), X_test), dim=1)

# Predictions using the estimated coefficients from Q1 (least squares)
y_pred_q1 = torch.matmul(X_test, theta_hat)

# Predictions using the trained neural network (Q4)
with torch.no_grad(): # Ensure no gradient computation during prediction
    y_pred_q4 = mySimpleNet(torch.tensor(test_set['inputs'], dtype=torch.float32))

# Calculate Mean Squared Error (MSE) for both methods
mse_q1 = nn.MSELoss()(y_pred_q1, y_test)
mse_q4 = nn.MSELoss()(y_pred_q4, y_test)

# Combinaison des biais et des poids
theta_estimated = np.concatenate((theta_estimated_bias.numpy(), theta_estimated_weights.numpy()))
print("Estimations des θk du modèle (via réseau de neurones):", theta_estimated)

# Comparaison avec les estimations obtenues dans la question 1 (moindres carrés)
# Supposons que vous avez déjà calculé theta_hat via l'équation normale dans la question 1

# Imprimer les estimations obtenues par la méthode des moindres carrés
print("Estimations des θk par méthode des moindres carrés :", theta_hat.numpy())

print(f"Mean Squared Error (Q1 - Least Squares): {mse_q1.item()}")
print(f"Mean Squared Error (Q4 - Neural Network): {mse_q4.item()}")

print("\nComparison:")
if mse_q1 < mse_q4:
    print("Least squares method (Q1) provides a better fit on the test set.")
elif mse_q1 > mse_q4:
    print("Neural network method (Q4) provides a better fit on the test set.")
else:
    print("Both methods provide similar fits on the test set.")

Estimations des θk du modèle (via réseau de neurones): [19.918228    0.02555635  0.02251558  0.06640711]
Estimations des θk par méthode des moindres carrés : [[16.558023 ]
 [ 3.2729847]
 [ 3.2948358]
 [ 6.687037 ]]
Mean Squared Error (Q1 - Least Squares): 3.993152618408203
Mean Squared Error (Q4 - Neural Network): 32.80284881591797

Comparison:
Least squares method (Q1) provides a better fit on the test set.


\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [37]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [38]:
# C'est un réseau de neurones convolutifs causal.
# Nb de paramètres dans self.Down1: (calcul "à la main")
# Nombre de parametre = (Cin*taille_du_kernel + 1)*Cout (1 pour le biais)
#Cin = 64
#Cout = 128
#taille_du_kernel = 3
#(64×3+1)×128=24,704

# Nb de paramètres au total:
# Définir le modèle
model = causalFCN()

# Calculer le nombre total de paramètres
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Nombre total de paramètres : {total_params}")


Nombre total de paramètres : 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

La taille du vecteur d'entrée est réduite par des couches de convolution causale et du pooling dans les couches Down_causal, notamment avec un stride de 5 dans down3. La taille est restituée par des convolutions transposées dans les couches Up_causal, qui augmentent la taille de la séquence tout en utilisant les informations des couches précédentes pour affiner la sortie.

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.


Le champ réceptif est augmenté par l'utilisation de convolutions dilatées et du down-sampling.



Voici le calcul détaillé du champ réceptif en sortie de `self.inc` :

1. **Première convolution** :
   - Taille du noyau \( k = 3 \)
   - Dilatation \( d = 1 \)
   Le champ réceptif \( r1 \) pour la première convolution est donné par la formule :

   r1 = (k - 1)\*d + 1 = (3 - 1)\*1 + 1 = 3

2. **Deuxième convolution** :
   - Taille du noyau \( k = 3 \)
   - Dilatation \( d = 1 \)
   Le champ réceptif \( r2 \) pour la deuxième convolution est également de 3, donc :                
   r2 = 3

3. **Champ réceptif total** :
   Comme les convolutions sont appliquées successivement, le champ réceptif total \( rtotal \) est la somme du champ réceptif de la première et de la seconde convolution, moins 1 :
   rtotal = r1 + (r2 - 1) = 3 + (3 - 1) = 5

####Résultat :
Le champ réceptif en sortie de `self.inc` est de **5**.

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [39]:
# Assuming 'model' and 'input_tensor1' are defined as in the previous code.

# Create two input tensors that differ only at index 0
input_tensor2 = copy.deepcopy(input_tensor1)
input_tensor2[0, 0, 0] = 1  # Change a single value in the input

# Perform the forward pass for both inputs
output1 = model(input_tensor1)
output2 = model(input_tensor2)


# Find the receptive field size
diff = torch.abs(output1 - output2)
receptive_field = torch.nonzero(diff.squeeze())
if len(receptive_field) > 0:
  print("Receptive field size (empirical):", receptive_field.max().item() + 1)
else:
  print("Could not determine the receptive field.")

Receptive field size (empirical): 10000


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



In [40]:
# Create two input tensors that differ only after index 5000
input_tensor3 = copy.deepcopy(input_tensor1)
input_tensor3[0, 0, 5001:] = 1  # Change values after index 5000

# Perform the forward pass for both inputs
output3 = model(input_tensor3)

# Compare the outputs at index 5000
diff_at_5000 = torch.abs(output1[0, 0, 5000] - output3[0, 0, 5000])
print(f"Difference at index 5000: {diff_at_5000}")

# The difference should be zero if the network is truly causal, which means that changing the inputs after
# index 5000 does not affect the output at index 5000.

# The part of the code in `Double_conv_causal` that guarantees causality is the padding operation:
#
# x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
#
# This padding adds zeros to the left side of the input tensor before the convolution is applied. The amount of padding
# is determined by the kernel size and the dilation.  By padding the input in this manner, convolutions only use
# information from the input that is temporally before the output element being calculated.
# This ensures that the output at a specific index doesn't depend on future inputs.

Difference at index 5000: 1.227647304534912


\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?